In [3]:
from langchain_core.tools import tool, StructuredTool
from typing import Optional


@tool
def multiply(a: Optional[int], b: int) -> int:
    """Multiply two numbers."""
    return a * b


# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'anyOf': [{'type': 'integer'}, {'type': 'null'}], 'title': 'A'}, 'b': {'title': 'B', 'type': 'integer'}}


---

In [160]:
"""Base class for Google Calendar tools."""

from __future__ import annotations

from typing import TYPE_CHECKING

from langchain_core.tools import BaseTool
from pydantic import Field

from langchain_google_community.gmail.utils import build_resource_service

if TYPE_CHECKING:
    # This is for linting and IDE typehints
    from googleapiclient.discovery import Resource  # type: ignore[import]
else:
    try:
        # We do this so pydantic can resolve the types when instantiating
        from googleapiclient.discovery import Resource
    except ImportError:
        pass

class GoogleCalendarBaseTool(BaseTool):
    api_resource: Resource = Field(default_factory=build_resource_service)
    
    @classmethod
    def from_api_resource(cls, api_resource: Resource) -> "GoogleCalendarBaseTool":
        """Create a tool from an api resource.

        Args:
            api_resource: The api resource to use.

        Returns:
            A tool.
        """
        return cls(api_resource=api_resource)

In [171]:
"""Create an event in Google Calendar."""

import re
from uuid import uuid4
from datetime import datetime
from typing import Any, Dict, List, Optional, Union, Type

from langchain_core.callbacks import CallbackManagerForToolRun

from pydantic import BaseModel, Field


def get_current_datetime() -> str:
    """Get the current datetime."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

class CreateEventSchema(BaseModel):
    """Input for CalendarCreateEvent."""

    summary: str = Field(
        ...,
        description="The title of the event."
    )
    start_datetime: str = Field(
        default=get_current_datetime(),
        description=("The start datetime for the event in 'YYYY-MM-DD HH:MM:SS' format"
                     f"The current year is {get_current_datetime()[:4]}"
                     "If the event is all day type, then set the time to 'YYYY-MM-DD' format.")
    )
    end_datetime: str = Field(
        ...,
        description=("The end datetime for the event in 'YYYY-MM-DD HH:MM:SS' format"
                     "If the event is all day type, then set the time to 'YYYY-MM-DD' format, and it has to be one day after the start date.")
    )
    calendar_id: str = Field(
        default="primary", 
        description="The calendar id to create the event in."
    )
    timezone: Optional[str] = Field(
        default=None,
        description="The timezone of the event."
    )
    recurrence: Optional[Dict[str, Any]] = Field(
        default=None,
        description=("The recurrence of the event."
                     "The format is"
                     "{'FREQ': <'DAILY' or 'WEEKLY'>,"
                     "'INTERVAL': <number>,"
                     "'COUNT': <number or None>,"
                     "'UNTIL': <'YYYYMMDD' or None>,"
                     "'BYDAY': <'MO', 'TU', 'WE', 'TH', 'FR', 'SA', 'SU' or None>}"
                     "Can be used COUNT or UNTIL, but not both, set the other to None.")
    )
    location: Optional[str] = Field(
        default=None,
        description="The location of the event."
    )
    description: Optional[str] = Field(
        default=None,
        description="The description of the event."
    )
    attendees: Optional[List[str]] = Field(
        delault=None,
        description="The list of attendees for the event."
    )
    reminders: Union[None, bool, List[Dict[str, Any]]] = Field(
        default=None,
        description=("The reminders for the event."
                     "If reminders are needed but are not specific, then set to 'True'"
                     "If specified, then set as [{'method': 'email', 'minutes': <minutes>}]"
                     "Or set as [{'method': 'popup', 'minutes': <minutes>}]"
                     "Where <minutes> is the number of minutes before the event."
                     "60 minutes = 1 hour."
                     "60 * 24 = 1 day.")
    )
    conference_data: Optional[bool] = Field(
        default=None,
        description="Whether to include conference data."
    )
    color_id: Optional[str] = Field(
        default=None,
        description=("The color id of the event, None is for default."
                     "'1': Lavender"
                     "'2': Sage"
                     "'3': Grape"
                     "'4': Flamingo"
                     "'5': Banana"
                     "'6': Tangerine"
                     "'7': Peacock"
                     "'8': Graphite"
                     "'9': Blueberry"
                     "'10': Basil"
                     "'11': Tomato")
    )


class CalendarCreateEvent(GoogleCalendarBaseTool):
    """Tool that create a event in Google Calendar."""

    name: str = "create_calendar_event"
    description: str = (
        "Use this tool to create an event." 
        "The input must be the summary, start and end datetime for the event."
    )
    args_schema: Type[CreateEventSchema] = CreateEventSchema

    def __get_timezone(self, calendar_id: str) -> str:
        """Get the timezone of the calendar."""
        calendars = self.api_resource.calendarList().list().execute().get('items', [])
        if calendar_id == 'primary':
            return calendars[0]['timeZone']
        else: 
            for item in calendars:
                if item['accessRole'] != 'reader' and item['id'] == calendar_id:
                    return item['timeZone']
            
    def __is_all_day_event(self, start_datetime: str, end_datetime: str) -> bool:
        """Check if the event is all day."""
        try:
            datetime.strptime(start_datetime, "%Y-%m-%d")
            datetime.strptime(end_datetime, "%Y-%m-%d")
            return True
        except ValueError:
            return False

    def _prepare_event(
        self,
        summary: str,
        start_datetime: str,
        end_datetime: str,
        calendar_id: str = 'primary',
        timezone: Optional[str] = None,
        recurrence: Optional[Dict[str, Any]] = None,
        location: Optional[str] = None,
        description: Optional[str] = None, 
        attendees: Optional[List[str]] = None,
        reminders: Union[None, bool, List[Dict[str, Any]]] = None,
        conferenceData: Optional[bool] = None, 
        color_id: Optional[str] = None
    ) -> Dict[str, Any]:
        """Prepare the event body."""
        timezone = timezone if timezone else self.__get_timezone(calendar_id)

        try:
            if self.__is_all_day_event(start_datetime, end_datetime):
                start = {"date": start_datetime, "timeZone": timezone}
                end = {"date": start_datetime, "timeZone": timezone}
            else:
                date_object = datetime.strptime(start_datetime, "%Y-%m-%d %H:%M:%S")
                start = {"dateTime": date_object.astimezone().replace(microsecond=0).isoformat(), "timeZone": timezone}
                date_object = datetime.strptime(end_datetime, "%Y-%m-%d %H:%M:%S")
                end = {"dateTime": date_object.astimezone().replace(microsecond=0).isoformat(), "timeZone": timezone}
        except ValueError as error:
            raise ValueError("The datetime format is incorrect format", error)
        
        recurrence_data = None
        if recurrence:
            if isinstance(recurrence, dict):
                recurrence_data = ['RRULE:']
                for k, v in recurrence.items():
                    if v is not None:
                        recurrence_data.append(f"{k}={v};")
                recurrence_data = ''.join(recurrence_data)
        
        attendees_mails = []
        if attendees and isinstance(attendees, list):
            for attendee in attendees:
                valid = re.match(r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$', attendee)
                if not valid:
                    raise ValueError(f"Invalid email address: {attendee}")
                attendees_mails.append({"email": attendee})
        
        reminders_info = None
        if reminders: 
            if reminders is True:
                reminders_info = {"useDefault": True}
            elif isinstance(reminders, list):
                for reminder in reminders:
                    if 'method' not in reminder or 'minutes' not in reminder:
                        raise ValueError("The reminders must have 'method' and 'minutes' keys.")
                    if reminder['method'] not in ['email', 'popup']:
                        raise ValueError("The reminders method must be 'email' or 'popup'.")
                reminders_info = {
                    'useDefault': False,
                    "overrides": reminders 
                }
        else:
            reminders_info = {"useDefault": False}
        
        if conferenceData:
            conferenceData = {
                "createRequest": {
                    "requestId": str(uuid4()),
                    "conferenceSolutionKey": {
                        "type": "hangoutsMeet"
                    }
                }
            }
        
        return {
            "summary": summary,
            "location": location,
            "description": description,
            "start": start,
            "end": end,
            "recurrence": [recurrence_data], 
            "attendees": attendees_mails, 
            "reminders": reminders_info,
            "conferenceData": conferenceData, 
            "colorId": color_id
        }
    
    def _run(
        self,
        summary: str,
        start_datetime: str,
        end_datetime: str,
        calendar_id: str = 'primary',
        timezone: Optional[str] = None,
        recurrence: Optional[Dict[str, Any]] = None,
        location: Optional[str] = None,
        description: Optional[str] = None,
        attendees: Optional[List[str]] = None,
        reminders: Union[None, bool, List[Dict[str, Any]]] = None,
        conference_data: Optional[bool] = None, 
        color_id: Optional[str] = None,
        run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Run the tool."""
        try:
            body = self._prepare_event(summary=summary, 
                                       start_datetime=start_datetime, 
                                       end_datetime=end_datetime, 
                                       calendar_id=calendar_id,
                                       timezone=timezone,
                                       recurrence=recurrence,
                                       location=location, 
                                       description=description, 
                                       attendees=attendees, 
                                       reminders=reminders,
                                       conferenceData=conference_data, 
                                       color_id=color_id)
            
            conferenceVersion = 1 if conference_data else 0
            event = self.api_resource.events().insert(calendarId=calendar_id, 
                                                      body=body, 
                                                      conferenceDataVersion=conferenceVersion).execute()
            return event.get('htmlLink')
        except Exception as error:
            raise Exception(f"An error occurred: {error}")


In [172]:
"""Get the events in Google Calendar."""


from datetime import datetime
from typing import Any, Dict, List, Optional, Union, Type

from langchain_core.callbacks import CallbackManagerForToolRun

from pydantic import BaseModel, Field

from pytz import timezone


def get_current_datetime() -> str:
        """Get the current datetime."""
        return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

class GetEventsSchema(BaseModel):
    """Input for CalendarGetEvents."""

    min_datetime: Optional[str] = Field(
        default=get_current_datetime(),
        description=("The start datetime for the events in 'YYYY-MM-DD HH:MM:SS' format"
                     "The current year is 2024")
    )
    max_datetime: Optional[str] = Field(
        ...,
        description="The end datetime for the events in 'YYYY-MM-DD HH:MM:SS' format",
    )
    max_results: int = Field(
        default=10,
        description="The maximum number of results to return."
    )
    

class CalendarGetEvents(GoogleCalendarBaseTool):
    """Tool that get the events in Google Calendar."""
    name: str = "get_events"
    description: str = "Use this tool to list the events in the calendar."
    args_schema: Type[GetEventsSchema] = GetEventsSchema


    def __get_calendars_info(self) -> List[Any]:
        """Get the calendars info."""
        calendars = self.api_resource.calendarList().list().execute()
        return calendars['items']
    
    def __get_calendar_timezone(self, calendars_info: List, calendar_id: str) -> Optional[str]:
        """Get the timezone of the current calendar."""
        for cal in calendars_info:
            if cal['id'] == calendar_id:
                return cal['timeZone']
        return None

    def __get_calendars(self, calendars_info: List) -> List[str]:
        """Get the calendars IDs."""
        calendars = []
        for cal in calendars_info:
            if cal.get('selected', None):
                calendars.append(cal['id'])
        return calendars
    
    def _process_data_events(self, events_data: List[Dict[str, Any]]) -> List[Dict[str, str]]:
        """Process the data events."""
        simplified_data = []
        for data in events_data:
            # Extract relevant fields
            event_dict = {
            "id": data["id"],
            "htmlLink": data["htmlLink"],
            "summary": data["summary"],
            "creator": data["creator"]["email"],
            "organizer": data["organizer"]["email"],
            "start": data["start"]["dateTime"],
            "end": data["end"]["dateTime"],
            }
            simplified_data.append(event_dict)
        return simplified_data
    
    def _run(
        self,
        min_datetime: Optional[str],
        max_datetime: Optional[str] = None,
        max_results: int = 10,
        run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> List[Dict[str, str]]:
        """Run the tool."""
        try:
            # body_request = self._prepare_request(min_datetime, max_datetime, max_results)
            calendars_info = self.__get_calendars_info()
            calendars = self.__get_calendars(calendars_info)
            events = []
            timeMin = None
            timeMax = None
            for calendar in calendars:
                region_tz = timezone(self.__get_calendar_timezone(calendars_info, calendar))
                if min_datetime:
                    timeMin = region_tz.localize(datetime.strptime(min_datetime, "%Y-%m-%d %H:%M:%S")).isoformat()
                if max_datetime:
                    timeMax = region_tz.localize(datetime.strptime(max_datetime, "%Y-%m-%d %H:%M:%S")).isoformat()
                events_result = self.api_resource.events().list(
                    calendarId=calendar, 
                    timeMin=timeMin, 
                    timeMax=timeMax, 
                    maxResults=max_results, 
                    singleEvents=True, 
                    orderBy="startTime"
                ).execute()
                cal_events = events_result.get('items', [])
                events.extend(cal_events)
            return self._process_data_events(events)
        except Exception as error:
            raise Exception(f"An error occurred: {error}")


In [173]:
from __future__ import annotations

from typing import TYPE_CHECKING, List

from langchain_community.agent_toolkits.base import BaseToolkit
from langchain_core.tools import BaseTool
from pydantic import ConfigDict, Field

from langchain_google_community.gmail.utils import build_resource_service

if TYPE_CHECKING:
    # This is for linting and IDE typehints
    from googleapiclient.discovery import Resource  # type: ignore[import]
else:
    try:
        # We do this so pydantic can resolve the types when instantiating
        from googleapiclient.discovery import Resource
    except ImportError:
        pass


SCOPES = ["https://www.googleapis.com/auth/calendar"]

class GoogleCalendarToolkit(BaseToolkit):
    """Toolkit for interacting with GoogleCalendar."""

    api_resource: Resource = Field(default_factory=build_resource_service)

    model_config = ConfigDict(
        arbitrary_types_allowed=True,
    )

    def get_tools(self) -> List[BaseTool]:
        """Get the tools in the toolkit."""
        return [
            CalendarCreateEvent(api_resource=self.api_resource),
            CalendarGetEvents(api_resource=self.api_resource)
        ]

In [174]:
from langchain_google_community.gmail.utils import (
    build_resource_service,
    get_gmail_credentials,
)


# Can review scopes here https://developers.google.com/gmail/api/auth/scopes
# For instance, readonly scope is 'https://www.googleapis.com/auth/gmail.readonly'
credentials = get_gmail_credentials(
    token_file="../secrets/token.json",
    scopes=["https://www.googleapis.com/auth/calendar"],
    client_secrets_file="../secrets/credentials.json",
)
api_resource = build_resource_service(credentials=credentials, service_name='calendar', service_version='v3')
toolkit = GoogleCalendarToolkit(api_resource=api_resource)

In [175]:
tools = toolkit.get_tools()
tools

[CalendarCreateEvent(api_resource=<googleapiclient.discovery.Resource object at 0x128cac800>),
 CalendarGetEvents(api_resource=<googleapiclient.discovery.Resource object at 0x128cac800>)]

In [176]:
import os
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model=os.getenv('OPENAI_MODEL'), api_key=os.getenv('OPENAI_API_KEY'))

In [177]:
model_with_tools = model.bind_tools(tools)

In [178]:
from langchain_core.messages import HumanMessage

prompt = "Crea un evento de todo el dia del evento de carros para hoy"

response = model_with_tools.invoke([HumanMessage(content=prompt)])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'create_calendar_event', 'args': {'summary': 'Evento de Carros', 'start_datetime': '2024-10-25', 'end_datetime': '2024-10-26', 'attendees': None, 'reminders': True}, 'id': 'call_hwXb9H4ho8be2Ds9sj4xTEyz', 'type': 'tool_call'}]


---

In [179]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [184]:
example_query = "Haz una sesion de meet para mañana a las 10am"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for data in events:
    data["messages"][-1].pretty_print()

================================ Human Message =================================

Haz una sesion de meet para mañana a las 10am
================================== Ai Message ==================================
Tool Calls:
  create_calendar_event (call_aPLaSAlTX7AFabyZEbm4WL3w)
 Call ID: call_aPLaSAlTX7AFabyZEbm4WL3w
  Args:
    summary: Sesión de Meet
    start_datetime: 2024-10-26 10:00:00
    end_datetime: 2024-10-26 11:00:00
    attendees: None
    conference_data: True
================================= Tool Message =================================
Name: create_calendar_event

Error: Exception('An error occurred: The read operation timed out')
 Please fix your mistakes.
================================== Ai Message ==================================
Tool Calls:
  create_calendar_event (call_QbVja0chCo2P6oSkOiOQnnRO)
 Call ID: call_QbVja0chCo2P6oSkOiOQnnRO
  Args:
    summary: Sesión de Meet
    start_datetime: 2024-10-26 10:00:00
    end_datetime: 2024-10-26 11:00:00
    attendees: 